<a href="https://colab.research.google.com/github/Adarsh-hello/deepfake_detection/blob/main/deepfake_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok gdown


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [10]:
%%writefile app.py
import streamlit as st
import numpy as np
from PIL import Image
import os

try:
    import gdown
except ImportError:
    os.system('pip install gdown')
    import gdown

from tensorflow.keras.models import load_model

st.set_page_config(page_title="Deepfake Detection using CNN", page_icon="🧠", layout="centered")

st.title("🧠 Deepfake Detection - CNN Model (with Google Drive)")

MODEL_PATH = "deepfake_detection.h5"
DRIVE_FILE_ID = "1azb4GHaBau-7cLK_9na_emb2t7iBgtXN"

def download_model():
    if not os.path.exists(MODEL_PATH):
        url = f"https://drive.google.com/uc?id={DRIVE_FILE_ID}"
        gdown.download(url, MODEL_PATH, quiet=False)

uploaded_file = st.file_uploader("📤 Upload an image", type=["jpg", "jpeg", "png"])

@st.cache_resource
def load_cnn_model():
    download_model()
    cnn = load_model(MODEL_PATH)
    return cnn

try:
    cnn_model = load_cnn_model()
except Exception as e:
    st.error(f"❌ Error loading model: {e}")
    st.stop()

labels = ['Real', 'Fake']

def preprocess(img):
    img = img.resize((128, 128))
    img_array = np.array(img) / 255.0
    if img_array.shape[-1] == 4:
        img_array = img_array[..., :3]
    return np.expand_dims(img_array, axis=0)

if uploaded_file is not None:
    st.image(uploaded_file, caption="🖼 Uploaded Image", use_column_width=False, width=250)
    try:
        image = Image.open(uploaded_file)
        processed = preprocess(image)
        cnn_pred = cnn_model.predict(processed)[0]
        cnn_result = labels[np.argmax(cnn_pred)]

        st.markdown("### 🔍 Prediction Result:")
        st.success(f"🧠 CNN Prediction: {cnn_result} ({cnn_pred[np.argmax(cnn_pred)]*100:.2f}%)")

    except Exception as e:
        st.error(f"Prediction error: {e}")


Writing app.py


In [11]:
from pyngrok import ngrok
import time
import threading

# Start the Streamlit server in a thread
def start_streamlit():
    !streamlit run app.py

thread = threading.Thread(target=start_streamlit)
thread.start()

# Wait for server to properly start
time.sleep(5)

# Set auth token only once
ngrok.set_auth_token("2wJv717kfaZlPwHjKAAQMErod1t_5iYEcD4RaSpoLkHAwgz2i")

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print("🚀 Public URL:", public_url)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.200.87:8501

🚀 Public URL: NgrokTunnel: "https://f211-34-125-200-87.ngrok-free.app" -> "http://localhost:8501"
